In [11]:
import tensorflow as tf
x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
f = x*x*y+y+2

sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [12]:
# with tf.Session() as sess:
#     sess.run(x.initializer)
#     sess.run(y.initializer)
#     result = f.eval()

In [13]:
import tensorflow as tf  
init = tf.global_variables_initializer()  # init 노드 준비

with tf.Session() as sess:
    sess.run(x.initializer)
    sess.run(y.initializer)
    result = f.eval()
    
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [14]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()


True

In [15]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)

x2.graph is tf.get_default_graph   # why?

True


False

In [16]:
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [17]:
# eval 대신 run 사용하기
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In [18]:
# GPU 사용 코드: GPU버전 tensorflow를 설치해야함
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name = "y")

XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value = theta.eval()


In [19]:
import tensorflow as tf

n_epochs = 1000
learning_rate = 0.01

# X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")  # error!!
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X,theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch",epoch,"MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 625323.0
Epoch 100 MSE = nan
Epoch 200 MSE = nan
Epoch 300 MSE = nan
Epoch 400 MSE = nan
Epoch 500 MSE = nan
Epoch 600 MSE = nan
Epoch 700 MSE = nan
Epoch 800 MSE = nan
Epoch 900 MSE = nan


In [20]:
def my_func(a,b):
    z = 0
    for i in range(100):
        z = a*np.cos(z+i) + z * np.sin(b-i)
    return z

In [21]:
gradients = tf.gradients(mse, [theta])[0]

In [22]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# 다른 옵티마이저 사용하기
# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

In [23]:
# placeholder!
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A+ 5

with tf.Session() as sees:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6],[7,8,9]]})
    
print(B_val_1)
print(B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [24]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
X = tf.placeholder(tf.float32, shape=(None, n), name="y")

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, barch_size):
    [...] # 디스크로부터 데이터 적재하기
    return X_batch, y_batch # 어찌리턴?

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
    best_theta = theta.eval()

NameError: name 'X_batch' is not defined

In [25]:
[...]
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0), name="theta")
[...]
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        
        sess.run(trainig_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

NameError: name 'trainig_op' is not defined

In [26]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    [...]

saver = tf.train.Saver({"weights":theta})

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /tmp/my_model_final.ckpt
	 [[Node: save/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2_2/tensor_names, save/RestoreV2_2/shape_and_slices)]]

Caused by op 'save/RestoreV2_2', defined at:
  File "C:\Users\nsa18\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\nsa18\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\nsa18\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\nsa18\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\nsa18\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\nsa18\Anaconda3\lib

In [27]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,now)

In [28]:
mse_summary = tf.summary.scalar('MSE',mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [29]:
[...]
for batch_index in range(n_batches):
    X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
    if batch_index%10 ==0:
        summary_str = mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
    sess.run(trainig_op, feed_dict={X:X_batch, y:y_batch})
[...]

file_writer.close()

NameError: name 'X_batch' is not defined

In [30]:
import tensorflow as tf
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = "mse")

print(error.op.name)
print(mse.op.name)

loss_2/sub
loss_2/mse


In [31]:
# 모듈화
n_features = 3
X = tf.placeholder(tf.float32, shape = (None, n_features), name = "X")

w1 = tf.Variable(tf.random_normal((n_features,1)), name = "weights1")
w2 = tf.Variable(tf.random_normal((n_features,1)), name = "weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X,w1),b1,name="z1")
z2 = tf.add(tf.matmul(X,w2),b2,name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")

output = tf.add(relu1,relu2, name="output")

In [33]:
def relu(X):
    w_shape = (int(X.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape), name = "weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X,w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape = (None, n_features), name = "X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

def relu(X):
    with tf.name_scope("relu"):
        [...]

In [34]:
# 변수 공유
def relu(X, threshold):
    with tf.name_scope("relu"):
        [...]
        return tf.maximum(z, threshold, name="max")
    
threshold = tf.Variable(0.0, name = "threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

# 공유 변수 적용
def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        [...]
        return tf.maximum(z, relu.threshold, name="max")


TypeError: Input 'y' of 'Maximum' Op has type float32 that does not match type int32 of argument 'x'.

In [37]:
# get_variable() 사용하기: 공유변수가 존재하지 않을 때 새로 만들고, 이미 있을 경우 재사용함
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold",shape=(), initializer=tf.constant_initializer(0.0))
    
# 변수 재사용하기 (reuse=True)
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")


In [38]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")
    
# resue가 True로 설정되면 블록 안에서 다시 False로 되돌릴 수 없다. 또한 이 블록 안에서 다른 변수의 범위를 정의하면 reuse=True를 상송받음

In [39]:
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")  # 기존변수 사용
        [...]
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name = "X")
with tf.variable_scope("relu"):
    threshold=tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))

relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

ValueError: Variable relu/threshold already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\nsa18\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1470, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\nsa18\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2956, in create_op
    op_def=op_def)
  File "C:\Users\nsa18\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [40]:
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold",shape=(), initializer=tf.constant_initializer(0.0))
        [...]
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name = "X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))

output = tf.add_n(relus, name="output")

ValueError: Variable relu/relu/threshold does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?